# General equations 
$$
        \begin{align*}
            \frac{\partial {\bf u}}{\partial t} + \nabla \cdot \Gamma = f
        \end{align*} \\
        \begin{align*}
            {\bf u} &= \begin{bmatrix}
                p \\
                m \\
                c_f \\
                c_b
            \end{bmatrix}, \,
            \nabla = \frac{\partial}{\partial x}
        \end{align*} \\
        \begin{align*}
            \Gamma &= \begin{bmatrix}
                        0 & -D_m (1-\beta p) \partial_x m & -D_c \partial_x c_f & 0 
                    \end{bmatrix}^T \\
            f &= \begin{bmatrix}
                v \partial_x p - \alpha p c_b \\
                \alpha p c_b \\
                -k_s c_f p + \alpha p c_b \\
                v \partial_x c_b + k_s c_f p - \alpha p c_b
            \end{bmatrix} \\
        \end{align*} \\
        \begin{align*}
            v &= \left. a_{br} k_{br} m^2 p \right\vert_{x=0}  + \left. a_{gr}k_{gr} m p \right\vert_{x=0}
        \end{align*}
$$

With the boundaries both at $x=0$ and $x=L$:
$$
    \begin{align}
        &p = 0 \\
        &D_m (1-\beta p) \partial_x m =  0 \\
        &\frac{\partial c_f}{\partial x} = 0 \\
        &c_b = 0 
    \end{align}
$$


# Discretization
**Convention**: A subscript of $i$ indicates the spatial discretizetion, an upperscript of $n$ indicates the temporal discretization. A subscript with no upper script means the current timestamp ($n$).

### Equations

$$
\begin {align}
    p_i^{n+1} &= p_i + \frac{\Delta t}{2 \Delta x} \cdot v (p_{i+1} -p_{i-1}) - Depoly \\
    m_i^{n+1} &= m_i + D_m \frac{\Delta t}{2 \Delta x^2} \big[(2 - \beta (p_i + p_{i+1}))(m_{i+1}-m_i) - (2 - \beta (p_i + p_{i-1}))(m_{i}-m_{i-1}) \big] + Depoly \\
    c_{f,i}^{n+1} &= c_{f, i} + D_c \frac{\Delta  t}{\Delta x^2} (c_{f, i+1} + c_{f, i-1} - 2 c_{f, i}) - Bind + Depoly \\
    c_{b,i}^{n+1} &= c_{b, i} + v \frac{\Delta  t}{2 \Delta x} (c_{b, i+1} - c_{b, i-1}) + Bind - Depoly
\end{align}
$$

Where:
$$
\begin{align}
    Depoly &= \Delta t \, \alpha c_{b, i} p_i \\ 
    Bind &= \Delta t \, k_s c_{f, i} p_i \\ 
\end{align}
$$

### Boundaries
#### Right boundary
$$
    p_{M} = 0 \\
    m_{M} = m_{M-2} \\
    c_{f, M} = c_{f, M-2} \\
    c_{b, M} = 0 \\
$$

#### Left boundary
$$
\begin{align}
    p_{0} = 0 \\
    m_{0} = m_{2} \\
    c_{f, 0} = c_{f, 2} \\
    c_{b, 0} = 0 \\
\end{align}
$$

* Note that the boundary of $m$ in both cases is dependent on $(1-\beta p)$. If $(1-\beta p) \rightarrow 0$, then $\partial_x m \rightarrow \infty$ 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Constants

Dm = 8.4
Dc = 10
beta = 0.1
alpha = 0
k_s = 0.001
k_gr = 8.7
k_br = 2.16E-5
a_gr = 2
a_br = 2
l = 0.003


$$
    \begin{align*}
        depoly &= \Delta t \cdot \alpha c_{b,i} p_i \\
        bind &= \Delta t \cdot k_s c_{f,i} p_i
    \end{align*}

$$

$$
    \begin{align*}
        f = \begin{bmatrix}
            v_i \partial_x p - \alpha p c_b \\
            \alpha p c_b \\
            -k_s c_f p + \alpha p c_b \\
            v \partial_x c_b + k_s c_f p - \alpha p c_b
        \end{bmatrix} \\
    \end{align*} \\

$$

In [ ]:
def get_f(p, cf, cb, dt, dx, v, alpha):
    """ Calculates the f part of the discretizied equations

    Args:
        p (_type_): p
        cf (_type_): cf
        cb (function): cb
        dt (_type_): dt
        dx (_type_): dx
        v (_type_): v

    Returns:
        tuple: f_p[-1:1], f_m[-1:1], f_cf[-1:1], f_cb[-1:1]
    """
    global k_s

    depoly = (dt*alpha[1: -1])*cb[1:-1]*p[1:-1]
    bind = (dt*k_s)*cf[1:-1]*p[1:-1]


    f_p = v*dt/(2*dx) * (p[2:] - p[:-2]) - depoly
    f_m = depoly
    f_cf = depoly - bind
    f_cb = v*dt/(2*dx) * (cb[2:] - cb[:-2]) - f_cf
    
    return f_p, f_m, f_cf, f_cb


# get_f(np.random.random(100), np.random.random(100), np.random.random(100), 0.01, 0.1, 5)

In [ ]:
def solver(I, L, T, dx, dt, user_action=None, stability_safety_factor=1.0, alpha_func=0.5):
    """_summary_
get_f - A function that returns a list of 4 values according to ${\bf f}$. The spatial derivatives would be calculated based on regular finite difference, e.g. $\frac{r_p^{n, m+1} - r_p^{n, m}}{\Delta x}$
    Args:
        I (List[Callable]): Initial condition for all of the dependent variables (list of 4 callables) 
        L (float): Length of domain
        T (float):  Stop time for the simulation 
        dx (float): Space grid unit
        dt (float): Time unit
        user_action (Callable, optional): _description_. Defaults to None.
        stability_safety_factor (float, optional): _description_. Defaults to 1.0.

    Returns:
        _type_: _description_
    """

    global a_br, k_br, a_gr, k_gr, Dm, Dc, beta, l

    Nt = int(round(T/dt))
    t = np.linspace(0, Nt*dt, Nt+1)      # Mesh points in time

    # Find max(c) using a fake mesh and adapt dx to C and dt
    # c_max = max([c(x_) for x_ in np.linspace(0, L, 101)])
    # dx = dt*c_max/(stability_safety_factor*C)

    Nx = int(round(L/dx))
    x = np.linspace(0, L, Nx+1)          # Mesh points in space

    p = np.vectorize(I[0])(x)
    m = np.vectorize(I[1])(x)
    cf = np.vectorize(I[2])(x)
    cb = np.vectorize(I[3])(x)

    alpha = np.vectorize(alpha_func)(x)


    for n in t:
        v = (a_br*k_br*m[1] + a_gr*k_gr*m[1]) * (m[1]*p[1])

        if user_action:
            user_action(p=p, m=m, cf=cf, cb=cb, v=v, dx=dx, dt=dt, x=x, t=t, n=n)

        
        f = get_f(p, cf, cb, dt, dx, v, alpha)
        # Calculate boundaries
        p_L, m_L, cf_L, cb_L = 0, m[-3], cf[-3], 0
        p_0 = 0
        m_0 = m[2]
        cf_0 = cf[2]
        cb_0 = 0 

        # Calculate inner-points
        m_flux = Dm*dt/(2*dx**2) * \
                    ((2-beta*(p[1: -1] + p[2:]))*(m[2:] - m[1: -1]) - (2-beta*(p[1: -1] + p[:-2]))*(m[1: -1] - m[:-2]))
        # print(m_flux)
        p[1: -1] = p[1: -1] + f[0]
        m[1: -1] = m[1: -1] +  m_flux + f[1]
        cf[1: -1] = cf[1: -1] + Dc*dt/(dx**2) * (cf[2:] + cf[:-2] - 2*cf[1:-1]) + f[2]
        cb[1: -1] = cb[1: -1] + f[3]

        # Set boundaries

        p[-1], m[-1], cf[-1], cb[-1] = p_L, m_L, cf_L, cb_L
        p[0], m[0], cf[0], cb[0] = p_0, m_0, cf_0, cb_0

    return p, m, cf, cb, x, t


    
# I = [
#     lambda x: 0,
#     lambda x: 0,
#     lambda x: 0,
#     lambda x: 0,
# ]

# sol = solver(I, 5, 2, 0.1, 0.01)

In [ ]:
import math
from IPython import display
from time import sleep

I = [
    lambda p: 0.1 * (p < 0.15),
    lambda m: 0.5,
    lambda cf: 0.01,
    lambda cb: 0.05 * (cb < 0.15),
]

# fig = plt.figure()
v = np.array(0)
def append_v(**kwargs):
    global v
    v_timestamp = kwargs['v']
    v = np.append(v, v_timestamp)

def plot_m(**kwargs):
    n = kwargs['n']
    if not( n % 0.01 <= 1e-7):
        return
        
    x = kwargs['x']
    m = kwargs['m']
    plt.title(f"n={n}")
    plt.plot(x, m, '-r')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    sleep(0.1)
    plt.clf()

def log_numbers(**kwargs):
    parameters = ["p", "m", "cf", "cb", "n"]
    [print(f"{k}: {kwargs[k]}, ", end="") for k in parameters]
    print()

def aggregate_gen(arr):
    def aggregate(**kwargs):
        for k in arr:
            _sum = sum(kwargs[k])
            print(f"{k}: {_sum}")
            


    return aggregate

    
sol = solver(I, 1, 2, 0.01, 1e-6, user_action=plot_m, alpha_func=lambda y: 0.7*math.tanh(y))


$$
    r_p^{(n+1, m)} = r_p^{(n, m)} + \frac{\Delta t}{2 \Delta x} v_p^{(n,m)} (r_p^{(n, m+1)}-r_p^{(n, m-1)}) - \Delta t \alpha r_p^{(n,m)} r_{sb}^{(n,m)}
$$

Boundary conditions on $x=0$:
$$
    r_p^{(n, 0)} = \frac{k_{on, br} (r_m^{(n, 0)})^2 + k_{on, gr} r_m^{(n, 0)}}{a_{br}k_{on, br} (r_m^{(n, 0)})^2 + a_{gr}k_{on, gr} r_m^{(n, 0)}}
$$

Boundary conditions on $x=M \Delta x$:
$$
    r_p^{(n, M)} = r_p^{(n, M-1)}
$$

In [ ]:
# def update_p(p, c_b, m, v, dt, dx, depoly=None):
#     """ Updates r_p in-place

#     Args:
#         r_p (npt.ndarray 1XM): r_p at t=n*dt and x=m*dx for n in [1, N] and every m in [0, M]
#         r_sb (npt.ndarray 1XM): r_sb at t=n*dt and x=m*dx for n in [1, N] and every m in [0, M]
#         r_m (npt.ndarray 1XM): r_m at t=n*dt and x=m*dx for n in [1, N] and every m in [0, M]
#         v_p (npt.ndarray 1XM): v_p at t=n*dt and x=m*dx for n in [1, N] and every m in [0, M]
#         dt (float): time step
#         dx (float): x step
#         depoly (npt.ndarray 1XM): the term alpha*r_{sb}*r_p. If it is supplied then it is not calculated
#     """
    
#     global alpha, k_on_br, k_on_gr, a_br, a_gr
    
#     if depoly is None:
#         depoly = alpha*r_p[1: -1]*r_sb[1: -1]
    
#     # Inner points    
#     r_p[1: -1] = r_p[1: -1] + (dt/(2*dx))*v_p[1: -1]*(r_p[2:] - r_p[:-2]) - dt*depoly
    
#     # Boundary x=M*dx
#     r_p[-1] = r_p[-2]

#     # Boundary x=0
#     br = k_on_br*r_m[0]**2
#     gr = k_on_gr*r_m[0]
    
#     r_p[0] = (br + gr)/(a_br*br +a_gr*gr)

#     return

# r_p = np.zeros(M) + 1
# r_sb = np.zeros(M) - 1
# r_m = np.zeros(M) + 1
# v_p = np.zeros(M) - 1

# print(r_p, end="\n\n")
# update_p(r_p, r_sb, r_m, v_p, 0.1, 0.1)
# print(r_p)

$$
    r_m^{(n+1, m)} = r_m^{(n, m)} + horror + \Delta t \alpha r_p^{(n,m)} r_{sb}^{(n,m)}
$$

$$
    r_{s,f}^{(n+1, m)} = r_{s,f}^{(n, m)} + \Delta t (D_s \frac{r_{s, f}^{(n, m+1)} + r_{s, f}^{(n, m-1)} - 2 r_{s, f}^{(n, m)}}{\Delta x ^2} - k_{on,s} r_{s,f}^{(n, m)} r_p^{(n, m)} + \alpha r_p^{(n,m)} r_{sb}^{(n,m)})
$$

Boundary conditions on $x=0$:
$$
    r_{s, f}^{(n, 0)} = r_{s, f}^{(n, 1)}
$$

Boundary conditions on $x=M \Delta x$:
$$
    r_{s, f}^{(n, M)} = r_{s, f}^{(n, M-1)}
$$